In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
from pyoso import Client

load_dotenv()
OSO_API_KEY = os.environ['OSO_API_KEY']
client = Client(api_key=OSO_API_KEY)

In [2]:
# PARAMS
START ='2024-01-01'
END   ='2025-04-01'
OWNER ='metacraft-labs'
REPO  ='codetracer'

In [3]:
stringify = lambda arr: "'" + "','".join(arr) + "'"

df_devs = client.to_pandas(f"""
    SELECT DISTINCT from_artifact_name AS github_user
    FROM int_events__github
    WHERE
      time BETWEEN DATE '{START}' AND DATE '{END}'
      AND event_type = 'STARRED'
      AND to_artifact_namespace = '{OWNER}'
      AND to_artifact_name = '{REPO}'
""")

devs_list = df_devs['github_user'].tolist()
print(f"Found {len(devs_list)} devs with star events to the project.")

df_repos = client.to_pandas(f"""
   SELECT
    from_artifact_name AS github_user,
    to_artifact_namespace AS repo_owner,
    SUM(amount) AS num_events
  FROM int_events__github
  WHERE time BETWEEN DATE '{START}' AND DATE '{END}'
    AND event_type = 'COMMIT_CODE'
    AND from_artifact_name IN ({stringify(devs_list)})
    AND from_artifact_name != to_artifact_namespace
    AND to_artifact_namespace != '{OWNER}'
  GROUP BY 1,2
""")

repo_owners = set(df_repos['repo_owner'].unique())
print(f"\nFound up to {len(repo_owners)} GitHub orgs with commits from these developers.")

df_op_repos = client.to_pandas("""
  SELECT DISTINCT artifact_namespace
  FROM artifacts_by_project_v1 AS abp
  JOIN projects_by_collection_v1 AS pbc ON abp.project_id = pbc.project_id
  WHERE collection_name IN ('op-retrofunding-4', '8-2')
  AND artifact_source = 'GITHUB'
""")

op_repos = set(df_op_repos['artifact_namespace'].to_list())
print(f"\nFound {len(op_repos)} GitHub orgs associated with Optimism Onchain Builders.")

print("\nFound the following connections with Onchain Builders:")
for p in op_repos.intersection(repo_owners):
    print(f"- {p}")

Found 936 devs with star events to the project.
Found up to 861 GitHub orgs with commits from these developers.

Found 342 GitHub orgs associated with Optimism Onchain Builders.

Found the following connections with Onchain Builders:
- lidofinance
